# Generic Imports and Config

In [17]:
%tensorflow_version 1.x

import os
import glob
import h5py
import shutil 
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.io import wavfile
import tensorflow as tf

%reload_ext autoreload
%autoreload 2

tf.__version__

TensorFlow is already loaded. Please restart the runtime to change versions.


'1.15.2'

In [18]:
%cd /content/drive/My Drive/Applause-Noise-Reduction
os.getcwd()

/content/drive/My Drive/Applause-Noise-Reduction


'/content/drive/My Drive/Applause-Noise-Reduction'

In [0]:
import preprocess as preprocess

# config
preprocess.SUBSET = "test"
preprocess.OUTPUT = "./data/Songs/test_song"
preprocess.NOISE_DATAPATH = preprocess.DATA_PATH + '/Applause'
preprocess.NOISE_DATAPATH_TEST = preprocess.DATA_PATH + '/Applause_test'
preprocess.VORBIS_WINDOW = preprocess.vorbis_window(preprocess.WINDOW_LEN)

# Calculating Test song FFT 

In [0]:
def process_test_song_as_h5(song_path):
    sr, song = wavfile.read(song_path)
    song_len = len(song)

    noisy_song = np.zeros(song_len)
    noise = np.zeros(song_len)
   
    for sl in range(song_len // (preprocess.SLICE_DURATION * preprocess.SAMPLE_RATE)):
        # get slices
        song_slice = song[sl * preprocess.SAMPLE_RATE * preprocess.SLICE_DURATION: (sl + 1) * preprocess.SLICE_DURATION * preprocess.SAMPLE_RATE]
        noisy_song_slice, noise_slice = preprocess.get_noisy_signal(song_slice)

        # calculate fft magn and phase for each slice
        noisy_song_fft_magn = []
        noisy_song_fft_phase = []
        frame_count = len(noisy_song_slice) // preprocess.STEP - 1

        for i in range(frame_count):
            noisy_cur_window = noisy_song_slice[i * preprocess.STEP: i * preprocess.STEP + preprocess.WINDOW_LEN]
            noisy_magnitude, noisy_phase = preprocess.get_magn_phase(noisy_cur_window)
            noisy_song_fft_magn.append(noisy_magnitude)
            noisy_song_fft_phase.append(noisy_phase)

        # save as hdf5 files
        magn_file_path = preprocess.get_path(song_path, 'magnitude', sl)
        with h5py.File(magn_file_path, 'w') as hf:
            hf.create_dataset('dataset', data=np.asarray(noisy_song_fft_magn))

        phase_file_path = preprocess.get_path(song_path, 'phase', sl)
        with h5py.File(phase_file_path, 'w') as hf:
            hf.create_dataset('dataset', data=np.asarray(noisy_song_fft_phase))

        # get whole noise and noisy_song
        noise[sl * preprocess.SAMPLE_RATE * preprocess.SLICE_DURATION: (sl + 1) * preprocess.SLICE_DURATION * preprocess.SAMPLE_RATE] += noise_slice
        noisy_song[sl * preprocess.SAMPLE_RATE * preprocess.SLICE_DURATION: (sl + 1) * preprocess.SLICE_DURATION * preprocess.SAMPLE_RATE] += noisy_song_slice

    return song, noise, noisy_song

In [0]:
test_song_path = './data/Songs/test_song/Carlos Gonzalez - A Place For Us/mixture16.wav'
sr = 16000

In [0]:
song, noise, noisy_song = process_test_song_as_h5(test_song_path)

In [23]:
print("Song")
display(Audio(song, rate=sr))
fig, ax = plt.subplots(figsize=(18,7))
ax.plot(song, label="Song");
ax.plot(noise, label="Noise", alpha=0.5);
plt.legend(loc="upper right");

print("Noisy song")
display(Audio(noisy_song, rate=sr))
fig, ax = plt.subplots(figsize=(18,7))
ax.plot(noisy_song, label="Noisy song");
ax.plot(song, label="Song", alpha = 0.5);

plt.legend(loc="upper right");

Output hidden; open in https://colab.research.google.com to view.

# Estimating Test song

In [24]:
!python main.py --mode 'test_song' --estimated_fft_path './data/Songs/test_song' --estimated_song_path './results/RNN'

2020-04-07 09:51:34.010827: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "main.py", line 5, in <module>
    tf.app.flags.DEFINE_string('mode', 'train', 'whether to train/test/preprocess')
AttributeError: module 'tensorflow' has no attribute 'app'


In [0]:
estimated_song_path = './results/RNN/estimated_song.wav'
sr, estimated_song = wavfile.read(estimated_song_path)

In [26]:
print("Estimated Song")
display(Audio(estimated_song, rate=sr))
print("Song")
display(Audio(song, rate=sr))

fig, ax = plt.subplots(figsize=(18,7))
ax.plot(estimated_song, label="Estimated Song");
ax.plot(song, label="Song", alpha=0.5);
plt.legend(loc="upper right");

print("Noisy song")
display(Audio(noisy_song, rate=sr))
fig, ax = plt.subplots(figsize=(18,7))
ax.set_ylim([-35000, 35000])
ax.plot(estimated_song, label="Estimated Song");
ax.plot(noisy_song, label="Noisy song", alpha = 0.2);
ax.plot(song, label="Song", alpha=0.7);

plt.legend(loc="upper right");

Output hidden; open in https://colab.research.google.com to view.